In [1]:
from pathlib import Path
from tempfile import TemporaryDirectory

import torch
from loguru import logger
from mido import MidiFile

from yoshimidi import player
from yoshimidi.data.parse import midi_parsing, one_hot_parsing, token_parsing, track_parsing

pygame 2.5.0 (SDL 2.28.0, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [32]:
PATH_INDEX = 1
TRACK_INDEX = 11
# PATH_INDEX = 2
# TRACK_INDEX = 0
CHANNEL_INDEX = None
midi_path = list(Path("../out/dataset/01_raw/lmd_full/").rglob("*.mid"))[PATH_INDEX]

In [34]:
player.play(midi_path)

2023-08-02 12:19:36.506 | INFO     | yoshimidi.player:play:12 - Playing ../out/dataset/01_raw/lmd_full/9/94078e08bc697e5ec566640c527de726.mid


2023-08-02 12:19:42.533 | INFO     | yoshimidi.player:play:19 - Finishing due to interrupt


In [35]:
midi_file = MidiFile(midi_path)
logger.info(f"num tracks: {len(midi_file.tracks)}")
logger.info(f"track num messages: {[len(track) for track in midi_file.tracks]}")
midi_track = midi_file.tracks[TRACK_INDEX]
tempo = track_parsing.parse_tempo(midi_file)
ym_track = track_parsing.from_midi(midi_track, ticks_per_beat=midi_file.ticks_per_beat, tempo=tempo)
assert ym_track is not None
ym_tokens = [token_parsing.from_channel(channel) for channel in ym_track.channels.values()]
if CHANNEL_INDEX is not None:
    ym_tokens = [ym_tokens[CHANNEL_INDEX]]
ym_one_hot = one_hot_parsing.from_tokens(ym_tokens[0], device=torch.device("cpu"), dtype=torch.float32)

ym_track_recons = track_parsing.from_tokens(ym_tokens)
midi_file_recons = midi_parsing.from_tracks([ym_track_recons])

with TemporaryDirectory() as temp:
    midi_path_recons = Path(temp) / "recons.mid"
    midi_file_recons.save(midi_path_recons)
    player.play(midi_path_recons)

2023-08-02 12:19:43.687 | INFO     | __main__:<module>:2 - num tracks: 1
2023-08-02 12:19:43.688 | INFO     | __main__:<module>:3 - track num messages: [2832]
2023-08-02 12:19:43.743 | INFO     | yoshimidi.player:play:12 - Playing /var/folders/7w/66fh_d3s5hb0br9f7wtqww1h0000gn/T/tmpt5mmqbzv/recons.mid
2023-08-02 12:19:51.086 | INFO     | yoshimidi.player:play:19 - Finishing due to interrupt
